<h1>Вступительное задание для вакансии Junior Python Programmer (стажер)</h1>

3. На языке Python реализовать функцию, которая быстрее всего (по процессорным тикам) отсортирует данный ей массив чисел. Массив может быть любого размера со случайным порядком чисел (в том числе и отсортированным). Объяснить почему вы считаете, что функция соответствует заданным критериям.

In [ ]:
class queue1:
    def __init__(self):
        self.value = []
    
    def push